# 🔢 Mô hình SVM Nhận dạng Chữ số Viết tay (MNIST)

Notebook này triển khai đầy đủ quy trình xây dựng mô hình SVM để nhận dạng chữ số viết tay sử dụng bộ dữ liệu MNIST.

**Các bước chính:**
1. Chuẩn bị dữ liệu và tiền xử lý
2. Huấn luyện mô hình SVM
3. Đánh giá mô hình
4. Tối ưu hóa mô hình (GridSearch, PCA)
5. Xuất đầu ra cho hệ ensemble

**Lưu ý:** Để sử dụng GPU trên Google Colab, notebook này sử dụng thư viện RAPIDS cuML khi có sẵn.

## 🚀 Cài đặt và Import thư viện

In [ ]:
# Kiểm tra GPU trên Google Colab
import subprocess
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
    print(result.stdout)
    GPU_AVAILABLE = True
except:
    print("Không tìm thấy GPU")
    GPU_AVAILABLE = False

In [ ]:
# Cài đặt RAPIDS cuML nếu có GPU (cho Google Colab)
# Uncomment dòng dưới nếu chạy trên Google Colab với GPU
# !pip install cuml-cu11 --extra-index-url=https://pypi.nvidia.com

In [ ]:
# Import các thư viện cần thiết
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from time import time
import warnings
warnings.filterwarnings('ignore')

# Sklearn imports
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score,
    ConfusionMatrixDisplay
)
from sklearn.calibration import CalibratedClassifierCV

# Cố gắng import cuML cho GPU acceleration
try:
    from cuml.svm import SVC as cuSVC
    from cuml.preprocessing import StandardScaler as cuStandardScaler
    from cuml.decomposition import PCA as cuPCA
    CUML_AVAILABLE = True
    print("✅ cuML đã được import thành công - Sử dụng GPU acceleration!")
except ImportError:
    CUML_AVAILABLE = False
    print("⚠️ cuML không khả dụng - Sử dụng sklearn (CPU)")

# Joblib để lưu model
import joblib

print(f"\n📊 Cấu hình:")
print(f"   - GPU Available: {GPU_AVAILABLE}")
print(f"   - cuML Available: {CUML_AVAILABLE}")

## 📦 1. Chuẩn bị dữ liệu và Tiền xử lý

In [ ]:
# Tải dữ liệu MNIST
print("📥 Đang tải dữ liệu MNIST...")
start_time = time()

# Cách 1: Sử dụng fetch_openml (dữ liệu đã flatten sẵn thành 784 features)
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False, parser='auto')

print(f"✅ Tải xong trong {time() - start_time:.2f} giây")
print(f"\n📊 Thông tin dữ liệu:")
print(f"   - Shape của X: {X.shape}")
print(f"   - Shape của y: {y.shape}")
print(f"   - Số lượng lớp: {len(np.unique(y))}")
print(f"   - Các lớp: {np.unique(y)}")
print(f"   - Dtype của X: {X.dtype}")
print(f"   - Range của pixel: [{X.min()}, {X.max()}]")

In [ ]:
# Hiển thị một số ảnh mẫu
fig, axes = plt.subplots(2, 10, figsize=(15, 4))
fig.suptitle('Một số ảnh mẫu từ MNIST', fontsize=14)

for i, ax in enumerate(axes.flat):
    ax.imshow(X[i].reshape(28, 28), cmap='gray')
    ax.set_title(f'Label: {y[i]}')
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Chuyển đổi nhãn sang số nguyên
y = y.astype(int)

# Chuẩn hóa dữ liệu: chia cho 255 để đưa về [0, 1]
print("🔄 Chuẩn hóa dữ liệu...")
X = X.astype(np.float32) / 255.0

print(f"   - Dtype sau chuẩn hóa: {X.dtype}")
print(f"   - Range sau chuẩn hóa: [{X.min():.2f}, {X.max():.2f}]")

In [ ]:
# Chia tập train/test theo chuẩn MNIST (60k train, 10k test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=10000, 
    random_state=42,
    stratify=y  # Đảm bảo phân bố đều các lớp
)

print(f"📊 Chia dữ liệu:")
print(f"   - Train: {X_train.shape[0]} mẫu")
print(f"   - Test: {X_test.shape[0]} mẫu")

# Phân bố các lớp trong tập train
unique, counts = np.unique(y_train, return_counts=True)
print(f"\n📈 Phân bố lớp trong tập train:")
for label, count in zip(unique, counts):
    print(f"   Chữ số {label}: {count} mẫu ({count/len(y_train)*100:.1f}%)")

In [ ]:
# Tùy chọn: Sử dụng tập dữ liệu nhỏ hơn để thử nghiệm nhanh
USE_SUBSET = True  # Đặt False để dùng toàn bộ dữ liệu
SUBSET_SIZE = 10000  # Số mẫu train để thử nghiệm

if USE_SUBSET:
    print(f"⚡ Sử dụng tập con {SUBSET_SIZE} mẫu để thử nghiệm nhanh...")
    # Lấy mẫu ngẫu nhiên có phân tầng
    from sklearn.model_selection import StratifiedShuffleSplit
    
    sss = StratifiedShuffleSplit(n_splits=1, train_size=SUBSET_SIZE, random_state=42)
    for train_idx, _ in sss.split(X_train, y_train):
        X_train_subset = X_train[train_idx]
        y_train_subset = y_train[train_idx]
    
    print(f"   - Tập train subset: {X_train_subset.shape[0]} mẫu")
else:
    X_train_subset = X_train
    y_train_subset = y_train
    print(f"📊 Sử dụng toàn bộ {X_train.shape[0]} mẫu train")

## 🧠 2. Huấn luyện Mô hình SVM

In [ ]:
# Định nghĩa hàm huấn luyện SVM
def train_svm(X_train, y_train, kernel='rbf', C=1.0, gamma='scale', 
              use_pca=False, n_components=100, use_gpu=False):
    """
    Huấn luyện mô hình SVM với các tùy chọn.
    
    Parameters:
    -----------
    X_train : array-like
        Dữ liệu huấn luyện
    y_train : array-like
        Nhãn huấn luyện
    kernel : str
        Loại kernel ('rbf', 'linear', 'poly', 'sigmoid')
    C : float
        Hệ số regularization
    gamma : str or float
        Hệ số gamma cho kernel RBF
    use_pca : bool
        Có sử dụng PCA giảm chiều hay không
    n_components : int
        Số thành phần PCA
    use_gpu : bool
        Sử dụng GPU (cuML) nếu có
        
    Returns:
    --------
    pipeline : sklearn Pipeline
        Model đã huấn luyện
    """
    steps = []
    
    # Bước 1: Chuẩn hóa (StandardScaler)
    if use_gpu and CUML_AVAILABLE:
        steps.append(('scaler', cuStandardScaler()))
    else:
        steps.append(('scaler', StandardScaler()))
    
    # Bước 2: PCA (tùy chọn)
    if use_pca:
        if use_gpu and CUML_AVAILABLE:
            steps.append(('pca', cuPCA(n_components=n_components)))
        else:
            steps.append(('pca', PCA(n_components=n_components)))
    
    # Bước 3: SVM
    if use_gpu and CUML_AVAILABLE:
        # cuML SVC
        svm = cuSVC(kernel=kernel, C=C, gamma=gamma, probability=True)
    else:
        # sklearn SVC
        svm = SVC(kernel=kernel, C=C, gamma=gamma, probability=True, cache_size=1000)
    
    steps.append(('svc', svm))
    
    # Tạo pipeline
    pipeline = Pipeline(steps)
    
    # Huấn luyện
    print(f"🏋️ Bắt đầu huấn luyện SVM...")
    print(f"   - Kernel: {kernel}")
    print(f"   - C: {C}")
    print(f"   - Gamma: {gamma}")
    print(f"   - PCA: {use_pca} ({n_components} components)" if use_pca else f"   - PCA: {use_pca}")
    print(f"   - GPU: {use_gpu and CUML_AVAILABLE}")
    
    start_time = time()
    pipeline.fit(X_train, y_train)
    train_time = time() - start_time
    
    print(f"\n✅ Huấn luyện hoàn tất trong {train_time:.2f} giây")
    
    return pipeline, train_time

In [ ]:
# Huấn luyện mô hình SVM cơ bản với RBF kernel
print("="*60)
print("🎯 Huấn luyện SVM với RBF Kernel")
print("="*60)

model_rbf, time_rbf = train_svm(
    X_train_subset, y_train_subset,
    kernel='rbf',
    C=1.0,
    gamma='scale',
    use_pca=False,
    use_gpu=CUML_AVAILABLE
)

In [ ]:
# Huấn luyện mô hình SVM với Linear kernel để so sánh
print("\n" + "="*60)
print("🎯 Huấn luyện SVM với Linear Kernel")
print("="*60)

model_linear, time_linear = train_svm(
    X_train_subset, y_train_subset,
    kernel='linear',
    C=1.0,
    use_pca=False,
    use_gpu=CUML_AVAILABLE
)

In [ ]:
# Huấn luyện mô hình SVM với PCA
print("\n" + "="*60)
print("🎯 Huấn luyện SVM với RBF Kernel + PCA")
print("="*60)

model_pca, time_pca = train_svm(
    X_train_subset, y_train_subset,
    kernel='rbf',
    C=1.0,
    gamma='scale',
    use_pca=True,
    n_components=100,
    use_gpu=CUML_AVAILABLE
)

## 📊 3. Đánh giá Mô hình

In [ ]:
def evaluate_model(model, X_test, y_test, model_name="Model"):
    """
    Đánh giá mô hình và in ra các chỉ số.
    
    Parameters:
    -----------
    model : sklearn Pipeline
        Mô hình đã huấn luyện
    X_test : array-like
        Dữ liệu test
    y_test : array-like
        Nhãn test
    model_name : str
        Tên mô hình để hiển thị
        
    Returns:
    --------
    dict : Các chỉ số đánh giá
    """
    print(f"\n{'='*60}")
    print(f"📊 Đánh giá: {model_name}")
    print(f"{'='*60}")
    
    # Dự đoán
    start_time = time()
    y_pred = model.predict(X_test)
    predict_time = time() - start_time
    
    # Tính accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"\n🎯 Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"⏱️ Thời gian dự đoán: {predict_time:.4f} giây")
    
    # Classification report
    print(f"\n📋 Classification Report:")
    print(classification_report(y_test, y_pred, digits=4))
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    
    return {
        'accuracy': accuracy,
        'predict_time': predict_time,
        'y_pred': y_pred,
        'confusion_matrix': cm
    }

In [ ]:
# Đánh giá các mô hình
results_rbf = evaluate_model(model_rbf, X_test, y_test, "SVM RBF Kernel")
results_linear = evaluate_model(model_linear, X_test, y_test, "SVM Linear Kernel")
results_pca = evaluate_model(model_pca, X_test, y_test, "SVM RBF + PCA")

In [ ]:
# Vẽ Confusion Matrix
def plot_confusion_matrix(cm, title="Confusion Matrix"):
    """
    Vẽ ma trận nhầm lẫn.
    """
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='YlOrRd',
                xticklabels=range(10), yticklabels=range(10))
    plt.title(title, fontsize=14)
    plt.xlabel('Dự đoán', fontsize=12)
    plt.ylabel('Thực tế', fontsize=12)
    plt.tight_layout()
    plt.show()

# Vẽ confusion matrix cho mô hình tốt nhất (RBF)
plot_confusion_matrix(results_rbf['confusion_matrix'], "Ma trận Nhầm lẫn - SVM RBF Kernel")

In [ ]:
# So sánh các mô hình
comparison_df = pd.DataFrame({
    'Model': ['SVM RBF', 'SVM Linear', 'SVM RBF + PCA'],
    'Accuracy': [results_rbf['accuracy'], results_linear['accuracy'], results_pca['accuracy']],
    'Train Time (s)': [time_rbf, time_linear, time_pca],
    'Predict Time (s)': [results_rbf['predict_time'], results_linear['predict_time'], results_pca['predict_time']]
})

print("\n📊 So sánh các mô hình:")
print(comparison_df.to_string(index=False))

# Vẽ biểu đồ so sánh
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Accuracy
axes[0].bar(comparison_df['Model'], comparison_df['Accuracy'], color=['#3498db', '#e74c3c', '#2ecc71'])
axes[0].set_ylabel('Accuracy')
axes[0].set_title('So sánh Accuracy')
axes[0].set_ylim([0.9, 1.0])
for i, v in enumerate(comparison_df['Accuracy']):
    axes[0].text(i, v + 0.002, f'{v:.4f}', ha='center')

# Train time
axes[1].bar(comparison_df['Model'], comparison_df['Train Time (s)'], color=['#3498db', '#e74c3c', '#2ecc71'])
axes[1].set_ylabel('Thời gian (s)')
axes[1].set_title('So sánh Thời gian Huấn luyện')

# Predict time
axes[2].bar(comparison_df['Model'], comparison_df['Predict Time (s)'], color=['#3498db', '#e74c3c', '#2ecc71'])
axes[2].set_ylabel('Thời gian (s)')
axes[2].set_title('So sánh Thời gian Dự đoán')

plt.tight_layout()
plt.show()

In [ ]:
# Hiển thị một số ảnh bị phân loại sai
def show_misclassified(X_test, y_test, y_pred, n_samples=10):
    """
    Hiển thị các ảnh bị phân loại sai.
    """
    misclassified_idx = np.where(y_test != y_pred)[0]
    
    if len(misclassified_idx) == 0:
        print("Không có mẫu nào bị phân loại sai!")
        return
    
    n_show = min(n_samples, len(misclassified_idx))
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    fig.suptitle(f'Các mẫu bị phân loại sai ({len(misclassified_idx)} mẫu)', fontsize=14)
    
    for i, ax in enumerate(axes.flat):
        if i < n_show:
            idx = misclassified_idx[i]
            ax.imshow(X_test[idx].reshape(28, 28), cmap='gray')
            ax.set_title(f'Thực: {y_test[idx]}\nDự đoán: {y_pred[idx]}', 
                        color='red', fontsize=10)
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

show_misclassified(X_test, y_test, results_rbf['y_pred'])

## ⚙️ 4. Tối ưu hóa Mô hình (GridSearch)

In [ ]:
# GridSearch để tìm siêu tham số tốt nhất
# Lưu ý: GridSearch tốn thời gian, nên sử dụng tập dữ liệu nhỏ

print("🔍 Bắt đầu Grid Search...")
print("⚠️ Quá trình này có thể mất vài phút...\n")

# Tạo pipeline cho GridSearch
pipeline_grid = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(probability=True, cache_size=1000))
])

# Định nghĩa lưới tham số
param_grid = {
    'svc__C': [0.1, 1, 10],
    'svc__gamma': ['scale', 0.01, 0.1],
    'svc__kernel': ['rbf', 'linear']
}

# Sử dụng tập con nhỏ hơn cho GridSearch
n_grid_samples = min(5000, len(X_train_subset))
X_grid = X_train_subset[:n_grid_samples]
y_grid = y_train_subset[:n_grid_samples]

print(f"📊 Sử dụng {n_grid_samples} mẫu cho GridSearch")
print(f"📊 Số lượng kết hợp tham số: {len(param_grid['svc__C']) * len(param_grid['svc__gamma']) * len(param_grid['svc__kernel'])}")

# Thực hiện GridSearch
grid_search = GridSearchCV(
    pipeline_grid,
    param_grid,
    cv=3,
    n_jobs=-1,
    verbose=2,
    scoring='accuracy',
    return_train_score=True
)

start_time = time()
grid_search.fit(X_grid, y_grid)
grid_time = time() - start_time

print(f"\n✅ GridSearch hoàn tất trong {grid_time:.2f} giây")

In [ ]:
# Kết quả GridSearch
print("\n📊 Kết quả GridSearch:")
print(f"   - Best Score (CV): {grid_search.best_score_:.4f}")
print(f"   - Best Parameters: {grid_search.best_params_}")

# Hiển thị top 5 kết hợp tham số
results_df = pd.DataFrame(grid_search.cv_results_)
results_df = results_df.sort_values('rank_test_score')[[
    'params', 'mean_test_score', 'std_test_score', 'mean_train_score', 'rank_test_score'
]].head(10)

print("\n🏆 Top 10 kết hợp tham số:")
print(results_df.to_string(index=False))

In [ ]:
# Huấn luyện mô hình với tham số tốt nhất trên toàn bộ dữ liệu
print("\n🏋️ Huấn luyện mô hình tối ưu với tham số tốt nhất...")

best_params = grid_search.best_params_

model_best, time_best = train_svm(
    X_train_subset, y_train_subset,
    kernel=best_params['svc__kernel'],
    C=best_params['svc__C'],
    gamma=best_params['svc__gamma'] if 'svc__gamma' in best_params else 'scale',
    use_pca=False,
    use_gpu=CUML_AVAILABLE
)

# Đánh giá mô hình tối ưu
results_best = evaluate_model(model_best, X_test, y_test, "SVM Tối ưu (Best Params)")

In [ ]:
# Thử nghiệm với PCA và các số thành phần khác nhau
print("\n📊 Thử nghiệm với PCA:")

pca_components = [50, 100, 150, 200]
pca_results = []

for n_comp in pca_components:
    print(f"\n🔄 Huấn luyện với PCA n_components={n_comp}")
    
    model_pca_test, train_time = train_svm(
        X_train_subset, y_train_subset,
        kernel='rbf',
        C=best_params.get('svc__C', 1.0),
        gamma='scale',
        use_pca=True,
        n_components=n_comp,
        use_gpu=CUML_AVAILABLE
    )
    
    # Đánh giá
    y_pred = model_pca_test.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    pca_results.append({
        'n_components': n_comp,
        'accuracy': accuracy,
        'train_time': train_time
    })
    print(f"   Accuracy: {accuracy:.4f}")

# Hiển thị kết quả PCA
pca_df = pd.DataFrame(pca_results)
print("\n📊 So sánh các cấu hình PCA:")
print(pca_df.to_string(index=False))

In [ ]:
# Vẽ biểu đồ ảnh hưởng của PCA
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Accuracy vs PCA components
axes[0].plot(pca_df['n_components'], pca_df['accuracy'], 'bo-', markersize=8)
axes[0].set_xlabel('Số thành phần PCA')
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Accuracy vs Số thành phần PCA')
axes[0].grid(True, alpha=0.3)

# Train time vs PCA components
axes[1].plot(pca_df['n_components'], pca_df['train_time'], 'ro-', markersize=8)
axes[1].set_xlabel('Số thành phần PCA')
axes[1].set_ylabel('Thời gian huấn luyện (s)')
axes[1].set_title('Thời gian huấn luyện vs Số thành phần PCA')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 🎯 5. Mô hình Cuối cùng và Xuất Đầu ra cho Ensemble

In [ ]:
# Huấn luyện mô hình cuối cùng với toàn bộ dữ liệu (nếu muốn)
print("="*60)
print("🏆 Huấn luyện Mô hình Cuối cùng")
print("="*60)

# Sử dụng tham số tốt nhất từ GridSearch
final_model = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(
        kernel=best_params['svc__kernel'],
        C=best_params['svc__C'],
        gamma=best_params.get('svc__gamma', 'scale'),
        probability=True,
        cache_size=1000
    ))
])

print(f"\n📊 Cấu hình mô hình cuối cùng:")
print(f"   - Kernel: {best_params['svc__kernel']}")
print(f"   - C: {best_params['svc__C']}")
print(f"   - Gamma: {best_params.get('svc__gamma', 'scale')}")

# Huấn luyện trên tập train đầy đủ (subset)
start_time = time()
final_model.fit(X_train_subset, y_train_subset)
final_train_time = time() - start_time

print(f"\n✅ Huấn luyện hoàn tất trong {final_train_time:.2f} giây")

# Đánh giá cuối cùng
final_results = evaluate_model(final_model, X_test, y_test, "Mô hình Cuối cùng")

In [ ]:
# Xuất xác suất dự đoán cho Ensemble
print("\n📤 Xuất đầu ra cho Ensemble:")

# Lấy xác suất dự đoán
proba = final_model.predict_proba(X_test)
pred = final_model.predict(X_test)

print(f"\n📊 Shape của xác suất: {proba.shape}")
print(f"   - Mỗi hàng là một mẫu")
print(f"   - Mỗi cột là xác suất cho chữ số 0-9")

# Hiển thị mẫu đầu ra
print(f"\n📋 Ví dụ 5 mẫu đầu tiên:")
sample_output = pd.DataFrame(
    proba[:5],
    columns=[f'P(digit={i})' for i in range(10)]
)
sample_output['Predicted'] = pred[:5]
sample_output['Actual'] = y_test[:5]
print(sample_output.to_string(index=False))

In [ ]:
# Lưu đầu ra cho ensemble
print("\n💾 Lưu đầu ra...")

# Tạo DataFrame cho ensemble
ensemble_output = pd.DataFrame(proba, columns=[f'prob_digit_{i}' for i in range(10)])
ensemble_output['predicted_label'] = pred
ensemble_output['true_label'] = y_test

# Lưu ra file CSV
ensemble_output.to_csv('svm_predictions_for_ensemble.csv', index=False)
print("✅ Đã lưu: svm_predictions_for_ensemble.csv")

# Lưu xác suất dạng numpy array
np.save('svm_probabilities.npy', proba)
print("✅ Đã lưu: svm_probabilities.npy")

# Lưu nhãn dự đoán
np.save('svm_predictions.npy', pred)
print("✅ Đã lưu: svm_predictions.npy")

## 💾 Save và Load Model

In [ ]:
# ============================
# SAVE MODEL VỀ MÁY
# ============================

import joblib
from google.colab import files  # Chỉ dùng trên Google Colab

# 1. Lưu model vào file
model_filename = 'svm_digit_classifier.joblib'
joblib.dump(final_model, model_filename)
print(f"✅ Đã lưu model: {model_filename}")

# 2. Lưu predictions và probabilities
np.save('svm_probabilities.npy', proba)
np.save('svm_predictions.npy', pred)
ensemble_output.to_csv('svm_predictions_for_ensemble.csv', index=False)
print("✅ Đã lưu predictions")

# 3. Download về máy (Google Colab)
print("\n📥 Đang tải files về máy...")
try:
    files.download(model_filename)
    files.download('svm_probabilities.npy')
    files.download('svm_predictions.npy')
    files.download('svm_predictions_for_ensemble.csv')
    files.download('confusion_matrix.png')
    print("✅ Tải thành công!")
except:
    print("⚠️ Không thể tự động download. Vui lòng download thủ công từ Files panel bên trái.")

In [ ]:
# ============================
# LOAD MODEL VÀ SỬ DỤNG LẠI
# ============================

# Load model đã lưu
loaded_model = joblib.load('svm_digit_classifier.joblib')
print("✅ Đã load model thành công!")

# Test với một ảnh
test_idx = 0
test_image = X_test[test_idx:test_idx+1]

# Dự đoán
prediction = loaded_model.predict(test_image)[0]
probability = loaded_model.predict_proba(test_image)[0]

print(f"\n🔮 Dự đoán: {prediction}")
print(f"🎯 Nhãn thực tế: {y_test[test_idx]}")
print(f"📊 Độ tin cậy: {probability[prediction]:.4f}")
print(f"📈 Xác suất các lớp: {dict(zip(range(10), probability.round(4)))}")

## 📖 Code sử dụng model trong session mới

Copy code dưới đây để sử dụng model đã lưu trong một notebook/script mới:

In [ ]:
# ============================
# CODE SỬ DỤNG TRONG SESSION MỚI
# ============================
# Copy đoạn code này để sử dụng model đã lưu

"""
import joblib
import numpy as np
from google.colab import files

# Upload model file (nếu cần)
# uploaded = files.upload()

# Load model
model = joblib.load('svm_digit_classifier.joblib')

# Chuẩn bị ảnh đầu vào (28x28 pixels, giá trị 0-255)
# image = your_image.reshape(1, 784)  # Flatten về 1D
# image = image.astype(np.float32) / 255.0  # Chuẩn hóa

# Dự đoán
# prediction = model.predict(image)[0]
# probabilities = model.predict_proba(image)[0]

# print(f"Predicted digit: {prediction}")
# print(f"Confidence: {probabilities[prediction]:.4f}")
"""

print("📋 Code mẫu đã được in ở trên. Uncomment để sử dụng.")

In [ ]:
# Lưu mô hình
print("\n💾 Lưu mô hình...")

joblib.dump(final_model, 'svm_digit_classifier.joblib')
print("✅ Đã lưu mô hình: svm_digit_classifier.joblib")

# Hướng dẫn sử dụng mô hình đã lưu
print("\n📖 Hướng dẫn sử dụng mô hình đã lưu:")
print("""
# Load mô hình
import joblib
model = joblib.load('svm_digit_classifier.joblib')

# Dự đoán nhãn
predictions = model.predict(X_new)

# Dự đoán xác suất
probabilities = model.predict_proba(X_new)
""")

## 📊 Tổng kết

In [ ]:
# Tổng kết kết quả
print("="*60)
print("📊 TỔNG KẾT KẾT QUẢ")
print("="*60)

print(f"\n🎯 Mô hình cuối cùng:")
print(f"   - Accuracy: {final_results['accuracy']:.4f} ({final_results['accuracy']*100:.2f}%)")
print(f"   - Kernel: {best_params['svc__kernel']}")
print(f"   - C: {best_params['svc__C']}")
print(f"   - Gamma: {best_params.get('svc__gamma', 'scale')}")

print(f"\n📁 Các file đã lưu:")
print("   - svm_digit_classifier.joblib (mô hình)")
print("   - svm_predictions_for_ensemble.csv (đầu ra cho ensemble)")
print("   - svm_probabilities.npy (xác suất dự đoán)")
print("   - svm_predictions.npy (nhãn dự đoán)")

print(f"\n✅ Hoàn tất!")

In [ ]:
# Vẽ confusion matrix cuối cùng với định dạng đẹp
plt.figure(figsize=(12, 10))

# Normalize confusion matrix
cm = final_results['confusion_matrix']
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Vẽ heatmap
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues',
            xticklabels=range(10), yticklabels=range(10),
            cbar_kws={'label': 'Tỷ lệ'})

plt.title('Ma trận Nhầm lẫn (Normalized) - Mô hình SVM Cuối cùng', fontsize=14)
plt.xlabel('Dự đoán', fontsize=12)
plt.ylabel('Thực tế', fontsize=12)
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=150, bbox_inches='tight')
print("✅ Đã lưu: confusion_matrix.png")
plt.show()

## 🔧 Bonus: Hàm tiện ích để dự đoán ảnh mới

In [ ]:
def predict_digit(model, image):
    """
    Dự đoán chữ số từ ảnh.
    
    Parameters:
    -----------
    model : sklearn Pipeline
        Mô hình SVM đã huấn luyện
    image : array-like
        Ảnh đầu vào (28x28 hoặc 784,)
        
    Returns:
    --------
    dict : Kết quả dự đoán
    """
    # Flatten nếu cần
    if image.ndim == 2:
        image = image.reshape(1, -1)
    elif image.ndim == 1:
        image = image.reshape(1, -1)
    
    # Chuẩn hóa nếu cần (giả sử ảnh có giá trị 0-255)
    if image.max() > 1:
        image = image.astype(np.float32) / 255.0
    
    # Dự đoán
    pred = model.predict(image)[0]
    proba = model.predict_proba(image)[0]
    
    return {
        'prediction': pred,
        'confidence': proba[pred],
        'probabilities': proba
    }

# Test với một ảnh từ tập test
test_image = X_test[0]
result = predict_digit(final_model, test_image)

print(f"🔮 Dự đoán: {result['prediction']}")
print(f"📊 Độ tin cậy: {result['confidence']:.4f}")
print(f"🎯 Nhãn thực tế: {y_test[0]}")

# Hiển thị ảnh
plt.figure(figsize=(8, 4))

plt.subplot(1, 2, 1)
plt.imshow(test_image.reshape(28, 28), cmap='gray')
plt.title(f'Dự đoán: {result["prediction"]} (Thực tế: {y_test[0]})')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.bar(range(10), result['probabilities'])
plt.xlabel('Chữ số')
plt.ylabel('Xác suất')
plt.title('Phân bố xác suất')
plt.xticks(range(10))

plt.tight_layout()
plt.show()